# Test various ACFGs for Core-Stability and Popularity

In [1]:
from min_acfg import find_popular_structure, find_core_stable_structure

## Core Stability in Example 3.3

In [2]:
n = 6
N = range(n)
F = {
    0: [1,2],
    1: [0,2],
    2: [0,1,3],
    3: [2,4,5],
    4: [3,5],
    5: [3,4]
}

struct_eq = find_core_stable_structure(N, F, degree='EQ')
struct_al = find_core_stable_structure(N, F, degree='AL')

    
if struct_eq is None:
    print('EQ:', 'No core stable structure!')
else:
    print('EQ:', struct_eq)
    
if struct_al is None:
    print('AL:', 'No core stable structure!')
else:
    print('AL:', struct_al)

EQ: [[3, 4, 5], [0, 1, 2]]
AL: [[3, 4, 5], [0, 1, 2]]


## Popularity in Example 3.4

In [3]:
n = 10
N = range(n)
F = {
    0: [1],
    1: [0,2],
    2: [1,3],
    3: [2,4,7],
    4: [3,5],
    5: [4,6],
    6: [5],
    7: [3,8],
    8: [7,9],
    9: [8]
}

struct_sf = find_popular_structure(N, F, degree='SF', strict=False)
struct_eq = find_popular_structure(N, F, degree='EQ', strict=False)
struct_al = find_popular_structure(N, F, degree='AL', strict=False)


if struct_sf is None:
    print('SF:', 'No popular structure!')
else:
    print('SF:', struct_sf)
    
if struct_eq is None:
    print('EQ:', 'No popular structure!')
else:
    print('EQ:', struct_eq)
    
if struct_al is None:
    print('AL:', 'No popular structure!')
else:
    print('AL:', struct_al)


SF: No popular structure!
EQ: No popular structure!
AL: No popular structure!
